In [67]:
import numpy as np
from scipy.stats import norm

In [68]:
N = norm.cdf

In [69]:
# Price at t=0
s0 = 95

# volatility
sigma = 0.305

# cont. compounded rf
r = 0.0405

# time to maturity
T = 187
Tp = 187/250

# spec. time relevant in some q's
t1 = 68
t1p = 68/250

# spec. time relevant in some q's
t2 = 120
t2p = 120/250

# single day
dt = 1/250

# Q1.

Compute the price of a plain vanilla at-the-money European call option with maturity at time 𝑇.

In [70]:
# Calculate price of call option

def bs_call(S_, X_, r_, Tp_, vol_):
    # Calculate d1
    d1 = (np.log(S_/X_)+(r_+(vol_**2)/2)*Tp_)/(vol_*np.sqrt(Tp_))
    # Calculate d2
    d2 = d1 - vol_ * np.sqrt(Tp_)
    # Calculate call value
    call = S_ * N(d1) - X_ * np.exp(-r_ * Tp_) * N(d2)
    
    print("S: ", S_)
    print("K: ", X_)
    print("r: ", r_)
    print("Tp: ", Tp_)
    print("sigma: ", vol_)
    print("d1: ", d1)
    print("d2: ", d2)
    print("N(d1): ",N(d1))
    print("N(d2): ",N(d2))
    
    return call

In [71]:
# Calculate call value based on implied volatility found

bs_call(s0, s0, r, Tp, sigma)

S:  95
K:  95
r:  0.0405
Tp:  0.748
sigma:  0.305
d1:  0.24673604916224662
d2:  -0.017049280225117147
N(d1):  0.5974437471462545
N(d2):  0.49319865076976716


11.301391298201324

# Q2.

Compute the price of a European call option with maturity at time 𝑇, and 𝐾 = 𝑆𝑡1.

In [72]:
# Define function to construct price paths according to Ito's Lemma

def price_paths(S0_, r_, sigma_, T_, dt_, n_paths):
    paths = []
    for iteration in np.arange(0,n_paths,1):
        current_price = S0_
        iteration_path = [current_price]
        z = norm.rvs(size = T_)
        for step in np.arange(1, T_, 1):
            current_price = current_price * np.exp((r_ - ((sigma_**2)/2)) * dt_ + z[step-1] * sigma_ * np.sqrt(dt_))
            iteration_path.append(current_price)
        paths.append([iteration, iteration_path])
    return paths


In [73]:
# Search for average price at time T

def average_price(paths_, time):
    sum = 0
    for path_ in range(0, len(paths_)):
        sum += paths_[path_][1][time-1]
    return sum/len(paths_)

In [74]:
# Construct price paths (adjustable number of iterations)

iter = 2000
paths = price_paths(s0, r, sigma, T, dt, iter)

In [75]:
# Define function to calculate call payoff

def call_payoff(sT_, K_):
    return max(sT_ - K_, 0)

In [76]:
# To get the price of a call, we will simulate the payoff in each simulated path

def simulate_payoffs(paths_, t_, T_):
    payoff_list = []
    for path in paths_:
        pi = call_payoff(path[1][T_-1], path[1][t_-1])
        payoff_list.append(pi)
    return payoff_list


In [77]:
discount_payoffs = np.exp(-r*Tp)

In [78]:
q2 = np.average(simulate_payoffs(paths, t1, T))
print("Average Call value with expiration at T and K = St1: ", q2*discount_payoffs)

Average Call value with expiration at T and K = St1:  8.682779804803522


# Q3. 

Consider the option described in question 2. What is the impact on the price of such an option, if
the underlying asset were to pay a one-off dividend (i.e., non-recurring dividend) after time 0, but before 𝑡1?
Would it increase, decrease, or remain unchanged? 

In [79]:
# Simulate prices with one-off dividend

def price_paths_break_div(S0_, r_, sigma_, T_, dt_, break_at, div_, n_paths):
    paths = []
    for iteration in np.arange(0,n_paths,1):
        current_price = S0_
        iteration_path = [current_price]
        z = norm.rvs(size = T_)
        for step in np.arange(1, T_, 1):
            if step != break_at:
                current_price = current_price * np.exp((r_ - ((sigma_**2)/2)) * dt_ + z[step-1] * sigma_ * np.sqrt(dt_))
                iteration_path.append(current_price)
            else:
                current_price = (current_price - div_) * np.exp((r_ - ((sigma_**2)/2)) * dt_ + z[step-1] * sigma_ * np.sqrt(dt_))
                iteration_path.append(current_price)
        paths.append([iteration, iteration_path])
    return paths

In [80]:
iter = 2000
div_time = int(t1 / 2)
for dividend in np.arange(1,21,1):
    paths_div = price_paths_break_div(s0, r, sigma, T, dt, div_time, dividend, iter)
    print("Average call price with dividend = ", dividend, ": ",np.average(simulate_payoffs(paths_div, t1, T)) * discount_payoffs)
    

Average call price with dividend =  1 :  8.320895387238624
Average call price with dividend =  2 :  8.770645947102324
Average call price with dividend =  3 :  8.579622079950381
Average call price with dividend =  4 :  7.992790826037715
Average call price with dividend =  5 :  8.632773857088402
Average call price with dividend =  6 :  8.39604963309457
Average call price with dividend =  7 :  7.736236139035681
Average call price with dividend =  8 :  7.95851564276069
Average call price with dividend =  9 :  7.986922382266308
Average call price with dividend =  10 :  8.30938259631074
Average call price with dividend =  11 :  7.353383881318344
Average call price with dividend =  12 :  7.816761135560847
Average call price with dividend =  13 :  7.731504783364101
Average call price with dividend =  14 :  7.506951528838674
Average call price with dividend =  15 :  7.467448237860672
Average call price with dividend =  16 :  7.018728626583436
Average call price with dividend =  17 :  7.52792764

# Q4.

Compute the price of an at-the-money European call option with maturity at time 𝑇, which also gives you the right to reset the strike price at time 𝑡1, such that 𝐾 = 𝑆𝑡1.

In [82]:
# For each path
payoff_list = []
for path in paths:
    sT = path[1][-1]
    st1 = path[1][t1-1]
    # Calculate payoff with normal strike
    no_option = call_payoff(sT, s0)
    # Decide to change to St1
    if st1 < s0:
        option = call_payoff(sT, st1)
    else:
        option = no_option
    # Add payoff to payoff list
    payoff_list.append(max(no_option, option))
print(np.average(payoff_list)*discount_payoffs)

12.810317551252751


In [83]:
# Would just say it is more expensive than both the previous ones, since it has the premium of the option of shifting from one to the other.

# Q5.

Compute the price of an at-the-money European call option with maturity at time 𝑇, which also gives you the right to reset the strike price at time 𝑡1, such that 𝐾 = 𝑆𝑡2 (where 𝑡1 < 𝑡2).

In [84]:
# For each path
payoff_list = []
for path in paths:
    sT = path[1][-1]
    st1 = path[1][t1-1]
    st2 = path[1][t2-1]
    # Calculate payoff with normal strike
    no_option = call_payoff(sT, s0)
    # Calculate payoff with changing strike (will only change strike if St1 < S0)
    if st1 < s0:
        option = call_payoff(sT, st2)
    else:
        option = no_option
    # Add payoff to payoff list
    payoff_list.append(max(no_option, option))
print(np.average(payoff_list)*discount_payoffs)

12.399583605705086


In [85]:
# Small difference but price is smaller, caused by a higher uncertainty when choosing to trade from one strike to another

# Q6.

Compute the price of an at-the-money European call option with maturity at time 𝑇, which also gives you the right to reset the strike price at time 𝑡1, such that:

• 𝐾 = 100% ∗ 𝑆𝑡2 if 𝑆𝑡2 > 90% ∗ 𝑆𝑡1

• 𝐾 = 105% ∗ 𝑆𝑡2 if 𝑆𝑡2 ≤ 90% ∗ 𝑆𝑡1

In [86]:
# For each path
payoff_list = []
for path in paths:
    sT = path[1][-1]
    st1 = path[1][t1-1]
    st2 = path[1][t2-1]
    # Calculate payoff with normal strike
    no_option = call_payoff(sT, s0)
    # Calculate payoff with changing strike
    if st2 > st1*0.9 and st2 < s0:
        option = call_payoff(sT, st2)
    elif st2 <= st1*0.9 and st2*1.05 < s0:
        option = call_payoff(sT, st2*1.05)
    else:
        option = no_option
    # Add payoff to payoff list
    payoff_list.append(max(no_option, option))
print(np.average(payoff_list)*discount_payoffs)

12.405929773406298


In [87]:
# Price of option remains similar to before, showing that both options, while different, will provide similar payoffs.